<a href="https://colab.research.google.com/github/Hashwineey/STQD6324_Data_Management_Assignment01/blob/main/Part1_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **STQD6324 DATA MANAGEMENT: ASSIGNMENT 1**

## **UCI Adult Income Dataset - Data Cleaning and Preprocessing**

This notebook is focused on the data preparation and cleaning process for the UCI Adult Income Dataset.  
The raw dataset was obtained from https://archive.ics.uci.edu/dataset/2/adult

### **Purpose of this Google Colab Notebook is to:**
- Load raw files into pandas
- Clean column headers and remove empty rows/columns
- Handle missing or malformed data
- Save the cleaned dataset as `.csv` files for downstream analysis in Hive

**The cleaned outputs from this notebook will be used in:**
- **Hive/Pig queries (for structured data analysis)**
- **Python visualizations and insights**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import libraries
import pandas as pd
import numpy as np

In [3]:
# Load adult.csv to pandas
adult_df = pd.read_csv('/content/drive/MyDrive/STQD6324_Data Management/Assignment1_Adult Income/adult.csv', header=None, na_values='?', skipinitialspace=True)

In [4]:
# Dimension of the raw data
adult_df.shape

(32561, 15)

In [5]:
# Preview the dataset
adult_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
# Investigate the data type
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       30725 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       30718 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      31978 non-null  object
 14  14      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


### **1. Assign proper column names to the columns**
* Column names are assigned based on the details listed in the data repository: https://archive.ics.uci.edu/dataset/2/adult

In [7]:
adult_df.columns = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"
]

In [8]:
# Preview the df
adult_df.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


### **2. Remove whitespace from string columns**
* This would be beneficial when this dataset is uploaded to Hive for query and analysis.

In [9]:
adult_df = adult_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
adult_df.head()

<ipython-input-9-3a0ccb9a968c>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  adult_df = adult_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### **3. Handle missing values**

In [10]:
# Check for missing values
adult_df.isnull().sum()

,0
age,0
workclass,1836
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,1843
relationship,0
race,0
sex,0


The missing values will be replaced with "Unknown" for *workclass* and *occupation* columns and "Other" for *native_country* column.

In [11]:
adult_df['workclass'].fillna('Unknown', inplace=True)
adult_df['occupation'].fillna('Unknown', inplace=True)
adult_df['native_country'].fillna('Other', inplace=True)

<ipython-input-11-762f62d53aba>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  adult_df['workclass'].fillna('Unknown', inplace=True)
<ipython-input-11-762f62d53aba>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [12]:
# Verify the missing values
print("Nulls after fill:\n", adult_df.isnull().sum())

Nulls after fill:
 age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64


### **4. Drop duplicates**

In [13]:
# Check for duplicates
adult_df.duplicated().sum()

np.int64(24)

In [14]:
# There 24 duplicated rows. Let's view them to verify.
adult_df[adult_df.duplicated()]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
4881,25,Private,308144,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Mexico,<=50K
5104,90,Private,52386,Some-college,10,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K
9171,21,Private,250051,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,10,United-States,<=50K
11631,20,Private,107658,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,10,United-States,<=50K
13084,25,Private,195994,1st-4th,2,Never-married,Priv-house-serv,Not-in-family,White,Female,0,0,40,Guatemala,<=50K
15059,21,Private,243368,Preschool,1,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,50,Mexico,<=50K
17040,46,Private,173243,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
18555,30,Private,144593,HS-grad,9,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,Other,<=50K
18698,19,Private,97261,HS-grad,9,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,40,United-States,<=50K
21318,19,Private,138153,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,10,United-States,<=50K


In [15]:
# Drop the duplicates
adult_df = adult_df.drop_duplicates()

In [16]:
# Verify the duplicates
adult_df.duplicated().sum()

np.int64(0)

In [17]:
# Check the dimension
adult_df.shape

(32537, 15)

### **5. Standardize the categories for each categorical column**
The categorical columns are:
* workclass
* education
* marital_status
* occupation
* relationship
* race
* sex
* native_country
* income

In [18]:
# General standardization (lower case all the categories)
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation',
                    'relationship', 'race', 'sex', 'native_country', 'income']

for col in categorical_cols:
    adult_df[col] = adult_df[col].str.strip().str.lower()

In [19]:
# List the categories of workclass
adult_df['workclass'].unique()

array(['state-gov', 'self-emp-not-inc', 'private', 'federal-gov',
       'local-gov', 'unknown', 'self-emp-inc', 'without-pay',
       'never-worked'], dtype=object)

In [20]:
# Standardize workclass column
adult_df['workclass'] = adult_df['workclass'].replace({
    'self-emp-not-inc': 'self-employed',
    'self-emp-inc': 'self-employed',
    'without-pay': 'unemployed',
    'never-worked': 'unemployed',
    'state-gov': 'government',
    'local-gov': 'government',
    'federal-gov': 'government'
})

In [21]:
# List the categories of education
adult_df['education'].unique()

array(['bachelors', 'hs-grad', '11th', 'masters', '9th', 'some-college',
       'assoc-acdm', 'assoc-voc', '7th-8th', 'doctorate', 'prof-school',
       '5th-6th', '10th', '1st-4th', 'preschool', '12th'], dtype=object)

In [22]:
# Standardize education column
education_mapping = {
    'bachelors': 'higher_edu',
    'masters': 'higher_edu',
    'doctorate': 'higher_edu',
    'prof-school': 'higher_edu',

    'some-college': 'associate_or_some_college',
    'assoc-acdm': 'associate_or_some_college',
    'assoc-voc': 'associate_or_some_college',

    'hs-grad': 'high_school_grad',
    '12th': 'high_school_grad',

    '11th': 'primary',
    '10th': 'primary',
    '9th': 'primary',
    '7th-8th': 'primary',
    '5th-6th': 'primary',
    '1st-4th': 'primary',

    'preschool': 'no_schooling'
}

adult_df['education_grouped'] = adult_df['education'].map(education_mapping)


In [23]:
# Verify education column
adult_df['education_grouped'].unique()

array(['higher_edu', 'high_school_grad', 'primary',
       'associate_or_some_college', 'no_schooling'], dtype=object)

In [24]:
# List of marital_status
adult_df['marital_status'].unique()

array(['never-married', 'married-civ-spouse', 'divorced',
       'married-spouse-absent', 'separated', 'married-af-spouse',
       'widowed'], dtype=object)

In [25]:
# Standardize marital_status column
adult_df['marital_status'] = adult_df['marital_status'].replace({
    'married-civ-spouse': 'married',
    'married-af-spouse': 'married',
    'divorced': 'separated',
    'separated': 'separated',
    'married-spouse-absent': 'separated',
    'never-married': 'single'
})

In [26]:
# List of occupation
adult_df['occupation'].unique()

array(['adm-clerical', 'exec-managerial', 'handlers-cleaners',
       'prof-specialty', 'other-service', 'sales', 'craft-repair',
       'transport-moving', 'farming-fishing', 'machine-op-inspct',
       'tech-support', 'unknown', 'protective-serv', 'armed-forces',
       'priv-house-serv'], dtype=object)

In [27]:
# Standardize occupation column
occupation_mapping = {
    'exec-managerial': 'white_collar',
    'prof-specialty': 'white_collar',

    'adm-clerical': 'admin_support_sales',
    'tech-support': 'admin_support_sales',
    'sales': 'admin_support_sales',

    'craft-repair': 'blue_collar',
    'machine-op-inspct': 'blue_collar',

    'handlers-cleaners': 'manual_labor',
    'transport-moving': 'manual_labor',
    'farming-fishing': 'manual_labor',

    'protective-serv': 'security_or_military',
    'armed-forces': 'security_or_military',

    'priv-house-serv': 'service',
    'other-service': 'service',

    'unknown': 'unknown'
}

adult_df['occupation_grouped'] = adult_df['occupation'].map(occupation_mapping)

In [28]:
# Verify occupation column
adult_df['occupation_grouped'].unique()

array(['admin_support_sales', 'white_collar', 'manual_labor', 'service',
       'blue_collar', 'unknown', 'security_or_military'], dtype=object)

In [29]:
# List of relationship
adult_df['relationship'].unique()

array(['not-in-family', 'husband', 'wife', 'own-child', 'unmarried',
       'other-relative'], dtype=object)

In [30]:
# List of race
adult_df['race'].unique()

array(['white', 'black', 'asian-pac-islander', 'amer-indian-eskimo',
       'other'], dtype=object)

In [31]:
# List of sex
adult_df['sex'].unique()

array(['male', 'female'], dtype=object)

In [32]:
# List of native_country
adult_df['native_country'].unique()

array(['united-states', 'cuba', 'jamaica', 'india', 'other', 'mexico',
       'south', 'puerto-rico', 'honduras', 'england', 'canada', 'germany',
       'iran', 'philippines', 'italy', 'poland', 'columbia', 'cambodia',
       'thailand', 'ecuador', 'laos', 'taiwan', 'haiti', 'portugal',
       'dominican-republic', 'el-salvador', 'france', 'guatemala',
       'china', 'japan', 'yugoslavia', 'peru',
       'outlying-us(guam-usvi-etc)', 'scotland', 'trinadad&tobago',
       'greece', 'nicaragua', 'vietnam', 'hong', 'ireland', 'hungary',
       'holand-netherlands'], dtype=object)

In [33]:
# Standardize the column by grouping them into continents
country_mapping = {
    # North America
    'united-states': 'north_america',
    'canada': 'north_america',
    'puerto-rico': 'north_america',
    'outlying-us(guam-usvi-etc)': 'north_america',
    'mexico': 'north_america',

    # Central America & Caribbean
    'cuba': 'central_america',
    'jamaica': 'central_america',
    'honduras': 'central_america',
    'dominican-republic': 'central_america',
    'el-salvador': 'central_america',
    'guatemala': 'central_america',
    'nicaragua': 'central_america',
    'trinadad&tobago': 'central_america',

    # South America
    'columbia': 'south_america',
    'ecuador': 'south_america',
    'peru': 'south_america',

    # Asia
    'india': 'asia',
    'china': 'asia',
    'iran': 'asia',
    'japan': 'asia',
    'philippines': 'asia',
    'cambodia': 'asia',
    'thailand': 'asia',
    'laos': 'asia',
    'taiwan': 'asia',
    'vietnam': 'asia',
    'hong': 'asia',

    # Europe
    'england': 'europe',
    'germany': 'europe',
    'france': 'europe',
    'italy': 'europe',
    'poland': 'europe',
    'portugal': 'europe',
    'yugoslavia': 'europe',
    'scotland': 'europe',
    'greece': 'europe',
    'ireland': 'europe',
    'hungary': 'europe',
    'holand-netherlands': 'europe',

    # Africa (none in this dataset, but include 'other')
    'other': 'other'
}

In [34]:
adult_df['native_region'] = adult_df['native_country'].map(country_mapping)

In [35]:
# Validate native_region
adult_df['native_region'].unique()

array(['north_america', 'central_america', 'asia', 'other', nan, 'europe',
       'south_america'], dtype=object)

In [36]:
# Recheck unmapped region
unmapped = adult_df[adult_df['native_region'].isnull()]['native_country'].unique()
print(unmapped)


['south' 'haiti']


In [37]:
# Add unmapped country to country_mapping
country_mapping.update({
    'south': 'north_america',
    'haiti': 'central_america',
    'netherlands': 'europe'
})

In [38]:
# Remap the column
adult_df['native_region'] = adult_df['native_country'].map(country_mapping)
adult_df['native_region'].fillna('other', inplace=True)  # Just in case

<ipython-input-38-30d92ff1b802>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  adult_df['native_region'].fillna('other', inplace=True)  # Just in case


In [39]:
# Verify native_region
adult_df['native_region'].unique()

array(['north_america', 'central_america', 'asia', 'other', 'europe',
       'south_america'], dtype=object)

In [40]:
# List income
adult_df['income'].unique()

array(['<=50k', '>50k'], dtype=object)

### **6. Bin age**

In [41]:
# List of age
adult_df['age'].unique()

array([39, 50, 38, 53, 28, 37, 49, 52, 31, 42, 30, 23, 32, 40, 34, 25, 43,
       54, 35, 59, 56, 19, 20, 45, 22, 48, 21, 24, 57, 44, 41, 29, 18, 47,
       46, 36, 79, 27, 67, 33, 76, 17, 55, 61, 70, 64, 71, 68, 66, 51, 58,
       26, 60, 90, 75, 65, 77, 62, 63, 80, 72, 74, 69, 73, 81, 78, 88, 82,
       83, 84, 85, 86, 87])

In [42]:
# Define bins and labels
bins = [0, 18, 25, 35, 45, 60, 75, 100]
labels = ['<18', '18–25', '26–35', '36–45', '46–60', '61–75', '76+']

In [43]:
# Create age bins
adult_df['age_group'] = pd.cut(adult_df['age'], bins=bins, labels=labels, right=True, include_lowest=True)

In [44]:
# Preview df
adult_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,education_grouped,occupation_grouped,native_region,age_group
0,39,government,77516,bachelors,13,single,adm-clerical,not-in-family,white,male,2174,0,40,united-states,<=50k,higher_edu,admin_support_sales,north_america,36–45
1,50,self-employed,83311,bachelors,13,married,exec-managerial,husband,white,male,0,0,13,united-states,<=50k,higher_edu,white_collar,north_america,46–60
2,38,private,215646,hs-grad,9,separated,handlers-cleaners,not-in-family,white,male,0,0,40,united-states,<=50k,high_school_grad,manual_labor,north_america,36–45
3,53,private,234721,11th,7,married,handlers-cleaners,husband,black,male,0,0,40,united-states,<=50k,primary,manual_labor,north_america,46–60
4,28,private,338409,bachelors,13,married,prof-specialty,wife,black,female,0,0,40,cuba,<=50k,higher_edu,white_collar,central_america,26–35


### **7. Drop unnecessary columns**

In [45]:
# Most of the columns like 'age' and 'hours_per_week' are kept for predictive modelling.
# Only columns that are already grouped and cleaned are dropped.
adult_df.drop(columns=['education', 'occupation',
                       'native_country'], inplace=True)

In [46]:
# Preview the dataset
adult_df.head()

,age,workclass,fnlwgt,education_num,marital_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,income,education_grouped,occupation_grouped,native_region,age_group
0,39,government,77516,13,single,not-in-family,white,male,2174,0,40,<=50k,higher_edu,admin_support_sales,north_america,36–45
1,50,self-employed,83311,13,married,husband,white,male,0,0,13,<=50k,higher_edu,white_collar,north_america,46–60
2,38,private,215646,9,separated,not-in-family,white,male,0,0,40,<=50k,high_school_grad,manual_labor,north_america,36–45
3,53,private,234721,7,married,husband,black,male,0,0,40,<=50k,primary,manual_labor,north_america,46–60
4,28,private,338409,13,married,wife,black,female,0,0,40,<=50k,higher_edu,white_collar,central_america,26–35


### **Final Check**

In [47]:
print("Cleaned shape:", adult_df.shape)
print("Nulls:", adult_df.isnull().sum())

Cleaned shape: (32537, 16)
Nulls: age                   0
workclass             0
fnlwgt                0
education_num         0
marital_status        0
relationship          0
race                  0
sex                   0
capital_gain          0
capital_loss          0
hours_per_week        0
income                0
education_grouped     0
occupation_grouped    0
native_region         0
age_group             0
dtype: int64


In [ ]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32537 entries, 0 to 32560
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   age                 32537 non-null  int64 
 1   workclass           32537 non-null  object
 2   fnlwgt              32537 non-null  int64 
 3   education           32537 non-null  object
 4   education_num       32537 non-null  int64 
 5   marital_status      32537 non-null  object
 6   occupation          32537 non-null  object
 7   relationship        32537 non-null  object
 8   race                32537 non-null  object
 9   sex                 32537 non-null  object
 10  capital_gain        32537 non-null  int64 
 11  capital_loss        32537 non-null  int64 
 12  hours_per_week      32537 non-null  int64 
 13  native_country      32537 non-null  object
 14  income              32537 non-null  object
 15  education_grouped   32537 non-null  object
 16  occupation_grouped  32537 n

### **Save cleaned version**

In [48]:
adult_df.to_csv("adult_cleaned.csv", index=False)

### **Download the cleaned CSV**

In [49]:
from google.colab import files
files.download("adult_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>